## 下载iflytek数据

In [1]:
import os
os.system("wget https://storage.googleapis.com/cluebenchmark/tasks/iflytek_public.zip && \
  unzip iflytek_public.zip && \
  rm iflytek_public.zip")

0

In [2]:
import json
records = []
with open('train.json', 'r') as f:
    for line in f.readlines():
        records.append(json.loads(line))
len(records)

12133

In [3]:
import pandas as pd
df = pd.DataFrame(records)
total_num, set_num = len(df), len(df['sentence'].drop_duplicates())
duplicate_num = total_num - set_num
print(f'原iflytek条数{total_num}, 去重后条数{set_num}, 重复条数{duplicate_num}')

原iflytek条数12133, 去重后条数11425, 重复条数708


In [4]:
diff_duplicate_num = 0
new_df = pd.DataFrame()
for sentence, sub_df in df.groupby('sentence'):
    if len(sub_df) > 1:
        diffs = len(set(sub_df['label'].tolist()))
        if diffs > 1:
            diff_duplicate_num += diffs - 1
        else:
            new_df = new_df.append(sub_df.iloc[0:1, :])
    else:
        new_df = new_df.append(sub_df)
new_df = new_df.sort_index().reset_index(drop=True)
print(f'语料重复且不一致的条数{diff_duplicate_num}')

语料重复且不一致的条数356


In [5]:
print('新iflytek条数：' + str(len(new_df)))

新iflytek条数：11084


In [6]:
with open("fixed_train.json", "w") as f:
    for record in new_df.to_dict(orient='records'):
        f.write(json.dumps(record,ensure_ascii=False) + '\n')

## 用CLUE原始脚本跑评测

+ ./CLUE/baselines/models_pytorch/classifier_pytorch/run_classifier_iflytek.sh

+ 原训练数据集
    验证集dev: acc = 0.5986918045402078
+ 新训练数据集
    验证集dev: acc = 0.596383224317045
## 结论
+ 指标下降了0.0023